# Each of the markdown sections below will be a separate code file (function notebook)

In [1]:
## put all the modules we have used up here possibly (numpy,matplotlib, etc)
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import csv
import numpy as np

import random
import itertools

from collections import defaultdict
from sklearn.model_selection import train_test_split

In [4]:
## this imports all the functions in the corresponding code file

%run image_preprocessing.ipynb

In [5]:
%run Watershed_Method.ipynb

## Train the FCN (these images were already preprocessed and bounded using Label Studio (Just run the output of Diana's code with this block, so that we get the trained network out)

This might be 2 separate FCNs with the same architecture, or one that can train on both
2 separate identical is easier to code

In [ ]:
#%run FCN-Segmentation.ipynb

In [ ]:
#Apply the FCN to the rest of the dataset

In [3]:
## Set the folder path for the balanced dataset
## set the filename for the balanced data csv file


# Folder containing the image files and the corresponding CSV file
folder_path = "Glaucoma_Balanced_Dataset/JustRAIGS"
csv_file = "JustRAIGS_Train_labels_balance.csv"
csv_path = os.path.join(folder_path,csv_file)
print(csv_path)

# Read the CSV file containing the image filenames and classifications
data = pd.read_csv(csv_path)


Glaucoma_Balanced_Dataset/JustRAIGS\JustRAIGS_Train_labels_balance.csv


## do the preprocessing on the balanced dataset (cropping & resizing, hist eq, normalization) [possibly repeat this step and break it into 2 steps for the ability to upload on github)

if we include cropping in this step, we can just output the final cropped images, and this may keep us below the github 2 gb limit

In [5]:
# Specify the desired output size (e.g., 1956 x 1725)
target_width = 2048
target_height = 1024

#The number of images will be randomly shuffled through and images can't be selected twice (not put back)
num_images = 10


# Process images and get a list of processed images (as NumPy arrays)
processed_images = process_images_from_csv(data, folder_path, target_width, target_height, num_images)

# Print the number of processed images (just for testing purposes)
print(f"Number of processed images: {len(processed_images)}")

Number of processed images: 10


In [ ]:
## For testing purposes only
##ONLY RUN THIS FOR LESS THAN 15 IMAGES, IT WILL DISPLAY ALL THE IMAGES

##This code block will be useful in applying this output later for the FCN step if we want


# Example: Access and work with the processed images
for idx, img_array in enumerate(processed_images):
    # Convert BGR to RGB (assuming img_array is in BGR format)
    image_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

    # Display the processed image using matplotlib
    plt.imshow(image_rgb)
    plt.title(f"Processed Image {idx + 1}")  # Set title
    plt.axis('off')  # Turn off axis
    plt.show()


## Use trained FCN to make an cup and a disk on each image

In [ ]:
## FCN should export cropped images (write them to jpg so we have a saved step)

#make sure to keep the filenames exactly the same

## copy and paste the csv file with the classifications, or fix the code to be able to work with the images and csv files being in different folders

## calculate the CDR for each image, and determine whether it is above or below the threshold for glaucoma

## Validate the performance with the expert opinions provided in the dataset